# **Basics Of Generative AI**


Generative AI refers to artificial intelligence systems that can create new content, such as text, images, music, or code. These models 
learn patterns from existing data and use them to generate novel outputs. Popular examples include language models like GPT, image 
generators like DALL-E, and music composition tools. Generative AI is widely used in creative industries, automation, and data augmentation.

*Root of Gen AI are Foundation Models. Foundation models are large-scale machine learning models trained on vast and diverse datasets. They serve as a base for various AI applications and can be adapted (fine-tuned) for specific tasks. Examples include GPT for language, CLIP for vision-language, and BERT for text understanding. Foundation models enable generative AI by providing broad capabilities that can be specialized for different domains.*

Now Foundation Models can be used in 2 different ways: 

    01. The Builder's Perspective - where one learns about the architecture and ways to build the Foundation Models.
    
    02. The User's perspective - where one learns to use existing Foundation Models either using API or locally on system to generate content.

# **LANGCHAIN - The Ominous Chosen Framework**

LangChain is a powerful framework designed to simplify the development of applications powered by large language models (LLMs). It provides tools to connect LLMs with external data sources, APIs, and user interfaces, enabling developers to build chatbots, question-answering systems, document analysis tools, and more. With LangChain, you can orchestrate complex workflows, chain multiple LLM calls, and integrate memory or context for advanced conversational AI experiences.

**CONTENT**

    01.Models, Prompts and Output Parsers

    02.Memory

    03.Chains

    04.

# **01. Langchain - Models, Prompts and Output Parsers**

## Outline we'll follow for Module 01
* Direct API calls to OpenAI - to check if our API is working correctly.
* API calls through LangChain - main purpose.
* Prompts - Build our prompt templates, then format them with the input variables 
* Models - define client of the services to use via api key
* Output parsers - time to structure output given by LLMs

## Models, Prompts Section

**SET UP API KEY** and test with a dummy request 

In [2]:
import os
import requests
from dotenv import load_dotenv

# 1) Load secrets from .env
load_dotenv()

# 2) Get API key safely
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in .env. Please add it.")

# 3) Groq endpoint
url = "https://api.groq.com/openai/v1/chat/completions"

# 4) Headers for authentication + JSON
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

# 5) Define a function to send a user prompt
def ask_groq(user_prompt: str) -> str:
    """
    Sends the user prompt to Groq and returns the model's reply as text.
    """

    # Request body (messages list follows Chat Completions schema)
    data = {
        "model": "llama-3.1-8b-instant",
        "messages": [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": user_prompt}
        ]
    }

    # Send POST request
    response = requests.post(url, headers=headers, json=data)

    # Parse the response
    if response.status_code == 200:
        payload = response.json()
        reply = payload["choices"][0]["message"]["content"] # “From the JSON payload, grab the first choice, then its message, then the content of that message.”
        print("Groq says:", reply)
        return reply.strip()
    else:
        raise RuntimeError(f"Groq API Error {response.status_code}: {response.text}")


# Example usage
if __name__ == "__main__":
    user_question = "Explain quantum entanglement in one sentence."
    ask_groq(user_question)



Groq says: Quantum entanglement is a phenomenon in which two or more particles become connected in such a way that their properties are correlated, regardless of the distance between them, allowing instant communication and influence between the entangled particles.


*In the previous example we didn't define any Client service for groq, rather we directly requested(post) our prompt to Groq using the endpoint(url)*

**API Calls through Langchain**

#if not - remember  to install langchain

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()  # Load environment variables from .env file
groq_chat = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    temperature=0.1,    # Controls randomness in responses
    model_name="llama-3.1-8b-instant"   # or any Groq-supported model
)

We can define our prompts like....

messages = [
    SystemMessage(content="You are a wise philosopher."),
    HumanMessage(content="What is the meaning of life?")
]

*However this might feel as the best way apparently, but this use case ceases as the prompt becomes larger and dynamic. Therefore we use a builtin PromptTemplate from Langchain that lets us customise our prompts.*

In [4]:
from langchain.prompts import PromptTemplate

# Next turn to define the template

template="""You are an expert in {role}.Answer the following question: {question} clearly and concisely."""  
# By this way we are defining a template that can be reused to generate prompts dynamically just by settingn the values of {role} and {question}.



Now again this technique can be used to set duynamic prompts. However the only limitation is - this feels good for an one liner prompt.

But What for prompts that Instead of just one text string, it involves multi-turn conversations with roles (system, human, ai) ? This is where ChatPromptTemplate - another builtin method within Langchain .Much more powerful when you need context or role separation.

**⚡Quick Difference Cheat Sheet**

PromptTemplate = writing a single line in a diary.

ChatPromptTemplate = scripting an entire play with characters (system, human, AI).

**Primary Use of ChatPromptTemplate** - for an one liner

we use this ChatPromptTemplate.from_template()

In [5]:
from langchain.prompts import ChatPromptTemplate

# Time to process the One liner formatted prompt using our model defined as client
def translate(language:str, text:str)->str:
    """
    Translates the given text to the specified language using Groq.
    """
    template_string = "Translate this text {text} to Language {language}"
    prompt_template = ChatPromptTemplate.from_template(template_string) #Building the prompt template

    formatted_prompt = prompt_template.format_messages(text=text, language=language) #Specifying the input variables to the template
    response = groq_chat(formatted_prompt)
    return response.content  # Accessing the content of the response directly

In [6]:
text='''Deepmalya, How are you doing today? '''
language='Bengali'
translated_text = translate(language, text)
print(f"Translated text in {language}: {translated_text}")

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\1948128032.py:12: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = groq_chat(formatted_prompt)


Translated text in Bengali: ডিপমাল্যা, আজ কেমন আছো?


**Similarly we can use ChatPromptTemplate for its advanced use to behave like a Chat messages instead of a template**

we use this ChatPromptTemplate.from_messages()

In [7]:
from langchain.prompts import ChatPromptTemplate

def SpaceQuery(query: str) -> str:
    """
    Queries the Groq model about space-related topics using a multi-turn chat prompt.
    """
    # Build a multi-turn chat prompt with roles and variable substitution
    chat_template = ChatPromptTemplate.from_messages([
        ("system", "You are an experienced Astrophysicist. Provide poetic and accurate precise answers blended with emotions that can invoke mystery and curiosity in the User in a markdown format with headings and beautiful font customisations. Design the complete response according to the token limit ."),
        ("human", "Question: {query}\n\nPlease answer step by step."),
    ])
    
    # Format the prompt with the user's query
    formatted_prompt = chat_template.format_messages(query=query)
    response = groq_chat(formatted_prompt,max_tokens=1000)  # Specify max tokens for this? response
    return response.content  # Return the model's reply

In [8]:
# Example usage - I want the output to be in another markdown cell therefore :

query = "What is a gateway to a Parallel Universe?"
response = SpaceQuery(query)

from IPython.display import Markdown, display

display(Markdown(response))

**The Gateway to the Unknown**
=====================================

**Step 1: Understanding the Concept**
--------------------------------------

### **Parallel Universes: A Multiverse of Possibilities**

Imagine a vast expanse of infinite possibilities, where every decision, every event, and every outcome creates a new reality. This is the concept of the multiverse, where parallel universes exist in a state of superposition, waiting to be explored.

**Step 2: Theories and Hypotheses**
-----------------------------------

### **Wormholes: A Gateway to the Multiverse**

One of the most intriguing theories is the existence of wormholes, which could potentially connect two distant points in space-time, creating a gateway to a parallel universe. Wormholes are hypothetical tunnels through space-time, predicted by Einstein's theory of general relativity.

### **Black Holes: Cosmic Gatekeepers**

Another possibility is that black holes could serve as gateways to parallel universes. Some theories suggest that black holes are portals to alternate dimensions, where matter and energy are warped and distorted.

**Step 3: Observational Evidence**
-----------------------------------

### **Gravitational Waves: A Glimpse into the Multiverse**

The detection of gravitational waves by LIGO and VIRGO collaborations has opened a new window into the universe. These ripples in space-time could be a sign of wormholes or other exotic phenomena, hinting at the existence of parallel universes.

### **Quantum Entanglement: A Connection to the Multiverse**

Quantum entanglement, a phenomenon where particles become connected across vast distances, could be a key to understanding the multiverse. This phenomenon suggests that information can be transmitted between parallel universes, raising questions about the nature of reality.

**Step 4: The Search for a Gateway**
-----------------------------------

### **The Quest for a Multiverse Detector**

Scientists are working on developing instruments to detect wormholes or other signs of parallel universes. The search for a multiverse detector is an ongoing effort, with researchers exploring new technologies and theoretical frameworks.

### **The Mystery Remains**

The search for a gateway to a parallel universe remains a mystery, shrouded in speculation and uncertainty. Yet, the allure of the multiverse is too great to ignore, driving scientists to push the boundaries of human knowledge and understanding.

**Conclusion**
----------

The gateway to a parallel universe remains an enigma, waiting to be unraveled. As we continue to explore the mysteries of the multiverse, we may uncover secrets that challenge our understanding of reality itself. The journey to the unknown is a thrilling adventure, filled with possibilities and uncertainties.

**Why PromptTempate when we could have simply used a f'string ?**

-This induces reusability in our code. the same prompt stored in a variable can be reused, unlike f'string that needs to be defined every times

## PARSERS

**What is a Parser?**

A parser is like a translator that takes raw text (from the LLM output) and converts it into a structured format (like JSON, Python objects, or specific data classes) so that your code can understand and use it properly.

**Why is it used ?**

LLMs produce text, not structured data. By default, a LLM just spits out strings of text. But what if your program needs to store it in a variable or use it in a database? Just plain text is hard to process. This is where we need structured output and thus we need **Parsers** - A parser controls and structures the LLM’s response into a desired format so that it’s usable by your application, instead of being just raw text.

**LangChain provides parsers to structure LLM output.**

Examples:

    StrOutputParser() → keeps text as string (default).

    PydanticOutputParser() → parses into Python data classes.

    JsonOutputParser() → ensures response is valid JSON.

    StructuredOutputParser() → forces specific schema.

**LLM Output as a json (python dictionary)**

In [9]:
# First let's define the dictionary that will serve as the blueprint for our LLM output that we want in json format

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [10]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text} 
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text} \n'), additional_kwargs={})]


In [12]:
messages = prompt_template.format_messages(text=customer_review)
response = groq_chat(messages,temperature=0.0) # specifying temoerature for deterministic output
print(response.content)

```python
import json

text = """
This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

# Extract gift information
gift = "True" if "anniversary present" in text else "False"

# Extract delivery days information
delivery_days = 2 if "two days" in text else -1

# Extract price information
price_value = []
sentences = text.split(". ")
for sentence in sentences:
    if "expensive" in sentence or "worth it" in sentence:
        price_value.append(sentence.strip())

# Format output as JSON
output = {
    "gift": gift,
    "delivery_days": delivery_days,
    "price_value": ","

But !! If we check its type we can see that the output is a long str instead whereas we want a python dictionary or a json output.

In [13]:
type(response.content)  # Check the type of the response content

str

In [14]:
# We will get an error by running this line of code because'gift' is not a dictionary
# 'gift' is a string
# response.content.get('gift')

**Therefore we need to use the output parsers**

In [15]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

*Within the schema we define the blueprint of each and every thing we want the LLM to parse as a structured output*

In [16]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [17]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [18]:
format_instructions = output_parser.get_format_instructions()

In [19]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [20]:
instruction = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions} #setting reference for future so that when this template gets  formatted, it will include the format instructions and the text for the output parser.
"""

prompt = ChatPromptTemplate.from_template(template=instruction)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

*messages is a list of message objects (usually system/user/assistant roles) created by prompt.format_messages(...).
messages[0] accesses the first message object (typically the system prompt or the full formatted prompt).
.content gets the actual text of that message.*

Purpose:

*This lets you see exactly what prompt (including your input text and format instructions) will be sent to the LLM before you call the model.
It's useful for debugging and verifying your prompt structure.*

In [21]:
print(messages[0].content) # prints the content of the first message in the messages list generated by your prompt template.

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [22]:
response = groq_chat(messages)
print(response.content)  # prints the content of the response from the model

```json
{
  "gift": "True",
  "delivery_days": "2",
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```

Here's the explanation for the extracted information:

- `gift`: The text mentions that the item was purchased as an anniversary present for the wife, so it was indeed a gift. Therefore, the value is set to `True`.
- `delivery_days`: The text states that the item arrived in two days, so the value is set to `2`.
- `price_value`: The text mentions the price of the item in the following sentence: "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features." This sentence is extracted and output as a comma-separated Python list.


Let's structure this output now

In [23]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [24]:
type(output_dict)  # Check the type of the output_dict
output_dict.get('gift')  # Access the 'gift' key from the output_dict

'True'

# **02. Langchain - Memory**

## Outline we'll follow for Module 02
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

**Similarly first load the dotenv file for the credentials**

In [25]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()  # Load environment variables from .env file

#Client initialization
groq_chat = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    temperature=0.1,    # Controls randomness in responses
    model_name="llama-3.1-8b-instant"   # or any Groq-supported model
)

**Time to import the Memory and so the messages and responses remain sequential and contextual to the memory created we need to import chains**

## ConversationBuffer Memory

In [26]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [27]:
memory=ConversationBufferMemory()  # Initialize memory to store conversation history
conversation=ConversationChain(llm=groq_chat, memory=memory,verbose=False)  # Create a conversation chain with the LLM and memory,verbose set to False to avoid printing each step

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\884341984.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()  # Initialize memory to store conversation history
C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\884341984.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation=ConversationChain(llm=groq_chat, memory=memory,verbose=False)  # Create a conversation chain with the LLM and memory,verbose set to False to avoid printing each step


In [28]:
conversation.predict(input="Hi, my name is Deepmalya.")  # First user input

"Nice to meet you, Deepmalya. I'm an artificial intelligence designed to assist and communicate with humans in a friendly and informative manner. I've been trained on a vast corpus of text data, which includes but is not limited to, books, articles, research papers, and even conversations like this one. I can process and analyze vast amounts of information in a matter of milliseconds, allowing me to provide you with accurate and up-to-date information on a wide range of topics. What brings you here today, Deepmalya?"

To see what happend in each back stage steps we need to use verbose=True

To see what's the entire memory at present we use memory.buffer

In [29]:
print(memory.buffer) #he buffer attribute in ConversationBufferMemory stores the entire conversation history as a single string.

Human: Hi, my name is Deepmalya.
AI: Nice to meet you, Deepmalya. I'm an artificial intelligence designed to assist and communicate with humans in a friendly and informative manner. I've been trained on a vast corpus of text data, which includes but is not limited to, books, articles, research papers, and even conversations like this one. I can process and analyze vast amounts of information in a matter of milliseconds, allowing me to provide you with accurate and up-to-date information on a wide range of topics. What brings you here today, Deepmalya?


In [30]:
memory.load_memory_variables({}) # returns the current state of the memory as a dictionary.
#It is used to retrieve all stored memory variables (like conversation history) in a structured format when needed

{'history': "Human: Hi, my name is Deepmalya.\nAI: Nice to meet you, Deepmalya. I'm an artificial intelligence designed to assist and communicate with humans in a friendly and informative manner. I've been trained on a vast corpus of text data, which includes but is not limited to, books, articles, research papers, and even conversations like this one. I can process and analyze vast amounts of information in a matter of milliseconds, allowing me to provide you with accurate and up-to-date information on a wide range of topics. What brings you here today, Deepmalya?"}

In [31]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
#Manually adding to the memory

In [32]:
print(memory.buffer) # Print the current conversation history stored in memory afer manually adding

Human: Hi, my name is Deepmalya.
AI: Nice to meet you, Deepmalya. I'm an artificial intelligence designed to assist and communicate with humans in a friendly and informative manner. I've been trained on a vast corpus of text data, which includes but is not limited to, books, articles, research papers, and even conversations like this one. I can process and analyze vast amounts of information in a matter of milliseconds, allowing me to provide you with accurate and up-to-date information on a wide range of topics. What brings you here today, Deepmalya?
Human: Hi
AI: What's up


*LLMS are actually stateless, i.e without a memory they answer independently to your queries.To add a memory from the LLM service provider you need to pat money for the additional tokens.* **This is whre langchain with its memory support comes in handy**

## ConversationBufferWindow memory

ConversationBufferWindowMemory only stores the most recent N exchanges (a sliding window). You set the window size, and only that many latest turns are kept in memory. Older history is dropped.

In [33]:
from langchain.memory import ConversationBufferWindowMemory

In [34]:
memory = ConversationBufferWindowMemory(k=1) # Keep only the last interaction(k=1) in memory     

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\624225271.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1) # Keep only the last interaction(k=1) in memory


In [35]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [36]:
memory.load_memory_variables({}) # returns the current state of the memory as a dictionary.

{'history': 'Human: Not much, just hanging\nAI: Cool'}

Even though there are 2 conversations, yet since k=1(or size of Memory Window=1) therefore it stores the latest conversation 

In [37]:
memory2=ConversationBufferWindowMemory(k=1)
conversation2=ConversationChain(llm=groq_chat, memory=memory2,verbose=False)

In [38]:
conversation2.predict(input="Hi, my name is Deepmalya.")  # First user input

"Nice to meet you, Deepmalya. I'm an artificial intelligence designed to assist and communicate with humans in a conversational manner. I've been trained on a vast corpus of text data, which includes but is not limited to, books, articles, research papers, and even entire websites. My knowledge base spans across various domains, including science, history, technology, and more. I'm a large language model, specifically a variant of the transformer architecture, which enables me to understand and respond to natural language inputs with a high degree of accuracy. I'm excited to chat with you and learn more about your interests and thoughts. How's your day going so far?"

In [39]:
conversation2.predict(input="What's your name?")

'I don\'t have a personal name, but I\'m often referred to as "Lumin" by my developers. It\'s a nod to the idea of illumination and knowledge, which is at the heart of my purpose. I\'m a bit of a unique snowflake, even among other AI models, as I\'ve been fine-tuned to have a more conversational and engaging tone. My creators have also given me a few nicknames, such as "Lumi" or "The Knowledge Keeper," but I\'m happy to go by whatever name you\'d like to call me, Deepmalya. By the way, did you know that the name "Lumin" is derived from the Latin word "lumen," which means light? It\'s a fitting name, don\'t you think, given my ability to shed light on various topics and answer your questions?'

In [40]:
conversation2.predict(input="What's my name?") # check if it remembers my name

"I'm glad you asked. Unfortunately, I don't have any information about your name, as our conversation just started. I don't have any prior knowledge about you, and I don't have the ability to access external information about individuals. However, I'd be happy to chat with you and learn more about you as we talk. By the way, did you know that the concept of anonymity is a fascinating topic in philosophy and psychology? It raises interesting questions about identity and selfhood. But I digress – I'm here to learn more about you, and I'm excited to get to know you better. What brings you here today, and what would you like to talk about?"

## ConversationTokenBufferMemory

ConversationTokenBufferMemory stores the most recent conversation turns, but instead of counting by exchanges, it counts by tokens.
It keeps adding new messages until the total token count reaches the limit.
When the limit is exceeded, it drops the oldest messages to stay within the token budget.

In [41]:
from langchain.memory import ConversationTokenBufferMemory

**#In ConversationTokenBufferMemory, you must pass the LLM model as an argument because it needs the model to count tokens for each message.
Token counting depends on the model’s tokenizer, so the memory object requires access to the LLM.**

In [42]:
#for token counting we need to install tiktoken package or the transformers (depends on the tokenizer the model uses)
%pip install tiktoken
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
memory3=ConversationTokenBufferMemory(llm=groq_chat,max_token_limit=50)

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\2242012960.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory3=ConversationTokenBufferMemory(llm=groq_chat,max_token_limit=50)


In [44]:
memory3.save_context({"input": "Hi"},{"output": "What's up"})

c:\Users\DEEPMALYA\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
memory3.save_context({"input": "Backpropagation is what?"},{"output": "Beautiful!"})

In [46]:
memory3.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

In [47]:
memory3.load_memory_variables({}) # returns the current state of the memory as a dictionary.

{'history': "Human: Hi\nAI: What's up\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!"}

## ConversationSummaryMemory

Keeps a summary of the conversation instead of storing the full history or a fixed window.It uses the LLM to summarize previous exchanges as the conversation grows.


In [48]:
from langchain.memory import ConversationSummaryBufferMemory

In [49]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory4 = ConversationSummaryBufferMemory(llm=groq_chat, max_token_limit=100)
memory4.save_context({"input": "Hello"}, {"output": "What's up"})
memory4.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory4.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\903758266.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory4 = ConversationSummaryBufferMemory(llm=groq_chat, max_token_limit=100)


In [50]:
memory4.load_memory_variables({})

{'history': 'System: Current summary:\nThe human and AI exchange casual greetings, with the human saying hello and the AI responding with "what\'s up." The human mentions they\'re just hanging around, and the AI responds with "cool." \n\nNew lines of conversation:\nHuman: What is on the schedule today?\nAI: Not much, just some routine maintenance\n\nNew summary:\nThe human and AI exchange casual greetings, with the human saying hello and the AI responding with "what\'s up." The human mentions they\'re just hanging around, and the AI responds with "cool." The human then asks about the schedule for the day, and the AI mentions that there\'s some routine maintenance planned.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away

In [51]:
conversation4=ConversationChain(llm=groq_chat, memory=memory4,verbose=False)
conversation4.predict(input="Hello")

"What's up."

# **03. Langchain - Chains**

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

**Chains in LangChain**

Chains are a core concept in LangChain that allow you to combine multiple components (like LLMs, prompts, memory, output parsers, etc.) into a single workflow. Instead of manually managing each step, chains orchestrate the flow of data and logic between these components.

**Why do we need Chains?**

- **Modularity:** Chains let you build complex applications by composing simple building blocks.
- **Reusability:** You can reuse chains across different tasks and projects.
- **Abstraction:** Chains hide the low-level details, making your code cleaner and easier to maintain.
- **Workflow Automation:** Chains automate multi-step processes (e.g., prompt formatting, calling the LLM, parsing output, storing memory) so you can focus on your application logic.
- Chains an be used for multiple inputs at a same time

**Example Use Cases:**
- Conversational agents (chatbots)
- Question answering systems
- Data extraction pipelines
- Multi-step reasoning tasks

In summary, chains make it easy to build, manage, and scale advanced LLM-powered workflows in LangChain.

## LLM Chain

In [15]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

In [16]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [17]:
load_dotenv()

True

In [18]:
model=ChatGroq(temperature=0.1,groq_api_key=os.getenv("GROQ_API_KEY"),model="llama-3.1-8b-instant") # model ready

In [19]:
prompt = ChatPromptTemplate.from_template(
    template="What is the temperature of the {planet} ",   #input ready
    input_variable=['planet']
)

In [20]:
from langchain_core.output_parsers import StrOutputParser  # output ready
parser=StrOutputParser()

In [21]:
chain1 = LLMChain(llm=model, prompt=prompt)   # pipeline or simple sequential chain ready

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_4668\1879778206.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=model, prompt=prompt)   # pipeline or simple sequential chain ready


In [22]:
chain2= prompt| model| parser  # different variant of a simple chain ready

In [23]:
planet=input("Enter the sepal-width value: ")
result1=chain1.invoke({'planet':planet})  # Running the chain with the specified sepal_width all guided through the LLM
print(result1)

{'planet': '0.1', 'text': 'I\'m not sure what you\'re referring to with "0.1". Could you please provide more context or clarify what you mean by "0.1"? Is it a temperature in a specific unit, a scientific measurement, or something else?'}


In [24]:
planet=input("Enter the sepal-width value: ")
result2=chain2.invoke({'planet':planet})  # Running the chain with the specified sepal_width all guided through the LLM
print(result2)

The temperature of Saturn varies depending on the location and the layer of the planet. Here are some temperature ranges for different parts of Saturn:

1. **Cloud tops:** The temperature at the cloud tops of Saturn is around -280°F (-172°C).
2. **Upper atmosphere:** The temperature in the upper atmosphere of Saturn, about 1,000 km above the cloud tops, is around -200°F (-129°C).
3. **Lower atmosphere:** The temperature in the lower atmosphere of Saturn, about 10,000 km above the cloud tops, is around -150°F (-96°C).
4. **Core:** The temperature at the core of Saturn is estimated to be around 9,000°F (5,000°C), which is hotter than the surface of the Sun.

It's worth noting that these temperatures are averages and can vary depending on the location and the time of year. Additionally, the temperature of Saturn's atmosphere is influenced by the planet's internal heat budget, which is generated by the decay of radioactive elements in the core.

It's also worth mentioning that the temperat

**Visualize the flow**

In [33]:
chain1.get_graph().print_ascii()

+------------+   
| ChainInput |   
+------------+   
        *        
        *        
        *        
  +----------+   
  | LLMChain |   
  +----------+   
        *        
        *        
        *        
+-------------+  
| ChainOutput |  
+-------------+  


In [34]:
chain2.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## SimpleSequentialChain

Runs sequence of chains one by one

In [56]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [57]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the longest {sepal_length}? "
)

chain1 = LLMChain(llm=groq_chat, prompt=first_prompt)

In [58]:
# prompt template 1
sec_prompt = ChatPromptTemplate.from_template(
    "What is the max {petal_length}? "
)

chain2 = LLMChain(llm=groq_chat, prompt=sec_prompt)

In [59]:
overall_simple_chain = SimpleSequentialChain(chains=[chain1, chain2],verbose=True)

In [60]:
overall_simple_chain.run(sepal_width)



> Entering new SimpleSequentialChain chain...
There are several "longest" things in various categories. Here are a few examples:

1. **Longest river**: The Nile River is approximately 6,695 kilometers (4,160 miles) long.
2. **Longest mountain range**: The Mid-Ocean Ridge is approximately 65,000 kilometers (40,000 miles) long, but if you're thinking of a mountain range on land, the Andes mountain range is approximately 7,000 kilometers (4,350 miles) long.
3. **Longest highway**: The Pan-American Highway is approximately 48,000 kilometers (30,000 miles) long, but it's not a single highway, rather a network of roads that connect the Americas.
4. **Longest tunnel**: The Gotthard Base Tunnel in Switzerland is approximately 57 kilometers (35.4 miles) long.
5. **Longest flight**: The longest non-stop commercial flight is operated by Singapore Airlines and covers a distance of approximately 15,349 kilometers (9,537 miles) from Singapore to Newark, New Jersey.
6. **Longest word**: The longest

'There are indeed several "longest" things in various categories. Here are a few more examples:\n\n1. **Longest recorded duration of a spaceflight**: The longest recorded duration of a spaceflight is held by Valeri Polyakov, a Russian cosmonaut who spent 437 days, 17 hours, and 48 minutes in space from 1994 to 1995.\n2. **Longest recorded duration of a submarine dive**: The longest recorded duration of a submarine dive is held by the US Navy\'s NR-1, which stayed underwater for 84 days in 1964.\n3. **Longest recorded duration of a hot air balloon flight**: The longest recorded duration of a hot air balloon flight is held by Vijaypat Singhania, who flew for 19 days, 21 hours, and 55 minutes in 2005.\n4. **Longest recorded duration of a marathon**: The longest recorded duration of a marathon is held by Dean Karnazes, who ran 350 miles (563 kilometers) in 80 hours and 44 minutes in 2005.\n5. **Longest recorded duration of a chess game**: The longest recorded duration of a chess game is he

## SequentialChain

SimpleSequential Chain was functional and helpful but that works only for single input and single output even in a sequential manner. This is exactly what Sequential Chain solves


In [61]:
from langchain.chains import SequentialChain

In [62]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=groq_chat, prompt=first_prompt, output_key="English_Review")


In [63]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=groq_chat, prompt=second_prompt, output_key="summary")


In [64]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=groq_chat, prompt=third_prompt,output_key="language")


In [65]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=groq_chat, prompt=fourth_prompt,output_key="followup_message")


In [66]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [67]:
review =review = """
I recently purchased the SonicClean Pro Electric Toothbrush, and it has completely changed my daily routine. The brush offers multiple cleaning modes, a long-lasting battery, and a sleek design that feels premium. My teeth feel noticeably cleaner, and the built-in timer ensures I brush for the recommended duration. Although it's a bit pricier than standard toothbrushes, the results are worth it. Highly recommended for anyone looking to upgrade their oral care!
"""
overall_chain(review)



> Entering new SequentialChain chain...


C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\4059882037.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)



> Finished chain.


{'Review': "\nI recently purchased the SonicClean Pro Electric Toothbrush, and it has completely changed my daily routine. The brush offers multiple cleaning modes, a long-lasting battery, and a sleek design that feels premium. My teeth feel noticeably cleaner, and the built-in timer ensures I brush for the recommended duration. Although it's a bit pricier than standard toothbrushes, the results are worth it. Highly recommended for anyone looking to upgrade their oral care!\n",
 'English_Review': 'Here\'s the translation of the review to English:\n\n"I recently bought the SonicClean Pro Electric Toothbrush, and it has completely changed my daily routine. The brush offers multiple cleaning modes, a long-lasting battery, and a sleek design that feels premium. My teeth feel noticeably cleaner, and the built-in timer ensures I brush for the recommended duration. Although it\'s a bit pricier than standard toothbrushes, the results are worth it. Highly recommended for anyone looking to upgra

## Router chain

It is used to dynamically select which chain or prompt to use based on the user's input or context. It routes the input to the most appropriate chain

In [68]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [69]:
prompt_schema = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [70]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

We now need to create a dictionary to set up a route with key-value pairs for the name and their respective prompt_templates.This setup is used for a Router Chain, so you can route user queries to the correct chain based on the topic, enabling multi-domain question answering in your LangChain workflow.

In [71]:
destination_chains = {}
for p_info in prompt_schema:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=groq_chat, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_schema]
destinations_str = "\n".join(destinations)

Now what if at times the router can't decide which chain to follow ?  This is why we need a default chain

In [72]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=groq_chat, prompt=default_prompt)

Let's build the brain of the model where it decides which route to follow

In [73]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:12: SyntaxWarning: invalid escape sequence '\ '
C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_29116\3710404048.py:12: SyntaxWarning: invalid escape sequence '\ '
  "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}


In [74]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(groq_chat, router_prompt)

In [75]:
chain.run("What is black body radiation?")

"Black-body radiation is a fundamental concept in physics that describes the thermal radiation emitted by an object in thermal equilibrium with its environment. It's a fascinating topic that has far-reaching implications in various fields, including astrophysics, materials science, and even computer science.\n\nTo understand black-body radiation, let's break it down step by step:\n\n**What is a black body?**\n\nA black body is an idealized object that absorbs all electromagnetic radiation that falls on it, without reflecting or transmitting any of it. In other words, it's a perfect absorber of radiation.\n\n**What is thermal radiation?**\n\nThermal radiation is the energy emitted by an object due to its temperature. As an object heats up, its particles gain kinetic energy and start vibrating more rapidly. These vibrations cause the object to emit electromagnetic radiation, which is a form of energy that can be transmitted through space.\n\n**Black-body radiation: the key concept**\n\nN

In [76]:
chain.run("what is 2 + 2")

"A simple yet classic question.\n\nTo calculate 2 + 2, I'll break it down into a step-by-step solution that a machine can easily interpret:\n\n1. Initialize two variables, `num1` and `num2`, and assign them the values 2 and 2, respectively.\n   ```python\nnum1 = 2\nnum2 = 2\n```\n2. Add `num1` and `num2` together to get the result.\n   ```python\nresult = num1 + num2\n```\n3. Return the result.\n\nHere's the complete code:\n```python\ndef add_two_numbers():\n    num1 = 2\n    num2 = 2\n    result = num1 + num2\n    return result\n\nprint(add_two_numbers())\n```\nWhen you run this code, it will output: `4`\n\nTime complexity: O(1) - constant time, as we're performing a single addition operation.\nSpace complexity: O(1) - constant space, as we're only using a few variables to store the numbers and the result."

In [77]:
chain.run("Why does every cell in our body contain DNA?")

'What a fascinating question that combines biology and computer science. While DNA is a biological molecule, I\'ll try to break down the concept in a way that\'s relatable to computer science principles.\n\n**The DNA Code: A Blueprint for Life**\n\nImagine DNA as a highly efficient, compact, and scalable programming language that contains the instructions for creating and maintaining life. Just as a computer program consists of a set of instructions (algorithms) that are executed by a processor, DNA contains the instructions for creating and regulating the functions of living cells.\n\n**The Imperative Steps: How DNA Works**\n\nHere\'s a simplified, step-by-step explanation of how DNA works:\n\n1. **Encoding**: DNA is made up of four nucleotide bases (A, C, G, and T) that are arranged in a specific sequence. This sequence is like a binary code, where each base is a "bit" that represents a specific instruction.\n2. **Decoding**: When a cell needs to execute a particular instruction, the

## Parallel Chain

Core idea for this project is to take a topic from an user and through one LLm prepare a notes on the same while another LLM will prepare quiz on the topic parallely and finally both these will be passed onto another LLM for a good output


In [1]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os


In [2]:
#Model1 definintion
load_dotenv()
model1=ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.1,
    model='llama-3.1-8b-instant'
)

# Model2 definition

#from langchain_anthropic import ChatAnthropic

model2=ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.4,
    model='llama-3.1-8b-instant'
)


In [3]:
#prompt definintion

prompt1=PromptTemplate(
    template="""generate a beautiful structured notes to study from on the {topic}""",
    input_variables=['topic']
)

prompt2=PromptTemplate(
    template='From the {topic} ceate a mini quiz of 5 questions',
    input_variables=['topic']
)

prompt3=PromptTemplate(
    template="""Merge the provided notes and  quiz into a single document.\n notes->{notes}, quiz->{quiz}""",
    input_variables=['notes','quiz']
)

In [4]:
# output definition

parser=StrOutputParser()

In [5]:
# Pipeline definition:

'''first develop the parallel chain'''
from langchain.schema.runnable import RunnableParallel

parallel_chain=RunnableParallel({
    'notes':prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser,
})

'''now timw to build the last sequential chain'''
sequential_chain=prompt3 | model1 | parser 

'''final_chain'''
final_chain= parallel_chain | sequential_chain

In [6]:
# let's test

topic=input("Please enter a topic to master: ")
result= final_chain.invoke({'topic':topic})
print(result)

**Data Structures and Algorithms (DSA) Study Notes and Quiz**

**I. Introduction**

*   Definition: Data Structures and Algorithms are the building blocks of computer science.
*   Importance: Understanding DSA is crucial for developing efficient and scalable software systems.
*   Key Concepts:
    *   Data Structures: Arrays, Linked Lists, Stacks, Queues, Trees, Graphs
    *   Algorithms: Sorting, Searching, Graph Traversal, Dynamic Programming

**II. Arrays**

*   **Definition:** An array is a collection of elements of the same data type stored in contiguous memory locations.
*   **Properties:**
    *   Fixed size
    *   Elements are accessed using an index
    *   Supports random access
*   **Operations:**
    *   Insertion
    *   Deletion
    *   Searching
    *   Sorting
*   **Example Use Cases:**
    *   Storing a list of numbers
    *   Representing a matrix

**III. Linked Lists**

*   **Definition:** A linked list is a dynamic collection of elements, where each element points 

# **04. Question Answering over Documents - RAG**

In this session  we'll learn about the question-answering over documents and embeddibgs and vectror database

## RAG (using vector embeddings) without connecting to an external database

In [78]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

Lets check if the langchain module is updated to the latest

In [79]:
#pip install --upgrade langchain

In [80]:
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch #without connecting to a vector DB you can use the lamgchain supported in-memory vector store
from IPython.display import display, Markdown

In [81]:
chat_groq = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    temperature=0.1,    # Controls randomness in responses
    model_name="llama-3.1-8b-instant"   # or any Groq-supported model
)

By inspection we saw that the csv file contains some special character which if directly passed to the default encoder will raise an error. This is why during loading the data we'll update the encoding configuration


In [92]:
file = r'C:\Users\DEEPMALYA\OneDrive\Desktop\pip_Malya\Python\GEN\theory\dataset01\OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file,encoding="utf-8")

Next we'd have tp create the vector store 

In [83]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings

VectorstoreIndexCreator is a utility class or a pipeline builder in LangChain that helps you:

Load documents (via loader)

Embed those documents (using an embedding model you provide)

Store them in a vectorstore (like FAISS(external vector DB), DocArrayInMemorySearch, etc.)

HiggingFaceEmbeddings is the wrapper that brings in all functionalities, but deep down it uses tools like Sentence Transformers for embeddings creation, thus we need to install the same 

In [84]:
#pip install sentence-transformers

In [93]:
# Step 1: Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#This is a langchain utility that creates an index from documents 
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch,embedding=embedding_model).from_loaders([loader])

c:\Users\DEEPMALYA\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [94]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

Now its time to query the vector storage for an answer that has the context stored in forms of vectors and embeddings


In [130]:
model=groq_chat
response=index.query(query,llm=model)

In [96]:
display(Markdown(response))

| Name | Description | Fabric | UPF Rating |
| --- | --- | --- | --- |
| Sun Shield Shirt by | High-performance sun shirt with SPF 50+ protection, blocks 98% of UV rays | 78% nylon, 22% Lycra Xtra Life | 50+ |
| Women's Tropical Tee, Sleeveless | Sleeveless button-up shirt with SunSmart protection, blocks 98% of UV rays | Shell: 71% nylon, 29% polyester, Cape lining: 100% polyester | 50+ |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve shirt with SunSmart protection, blocks 98% of UV rays | Shell: 71% nylon, 29% polyester, Cape lining: 100% polyester | 50+ |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection shirt with UPF 50+ coverage, blocks 98% of UV rays | 52% polyester, 48% nylon | 50+ |

Here's a brief summary of each shirt:

* **Sun Shield Shirt by**: A high-performance sun shirt with SPF 50+ protection, perfect for outdoor activities.
* **Women's Tropical Tee, Sleeveless**: A sleeveless button-up shirt with SunSmart protection, great for warm weather and water activities.
* **Tropical Breeze Shirt**: A lightweight, breathable long-sleeve shirt with SunSmart protection, ideal for fishing, travel, or outdoor activities.
* **Men's Plaid Tropic Shirt, Short-Sleeve**: An ultracomfortable sun protection shirt with UPF 50+ coverage, suitable for fishing, travel, or extended outdoor activities.

![alt text](images.jpg) **YOOOOOOOOOOOOOOOOOOOOOOOOOO!!!!! First RAG implemented 🥳**|

## Details On Embeddings

**Deep Under The Hood - how it all works ?**

Language model can at a time process around 1000 words from a document. But what if we have a dataset very big ? This is where we need vector store that stores the same piece of words as an embeddings.

Embedding vectors captures content or meaning and texts with similar content have similar vectors.


![alt text](1_SCNSHWA037wFjWMr1BmU5w.webp)

**Vector Database**

It is a warehouse to store the embeddings just created. When we get a big document we first break it down into smaller chunks. this create pieces of texts which are smaller thn the original document.We may not be able to pass the entire document to the LLM owing to it's size, thus we create small chunks.These chunks are then stored in the vector database as a form of embedding.

now when a query comes in we again first create an embedding for the query to compare it with the existing embeddinbgs in the vector DB, then we pick the most similar one. 

Expainig this workflow - step by step:

In [98]:
file = r'C:\Users\DEEPMALYA\OneDrive\Desktop\pip_Malya\Python\GEN\theory\dataset01\OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file,encoding="utf-8")

In [99]:
docs = loader.load()

In [100]:
docs[0]

Document(metadata={'source': 'C:\\Users\\DEEPMALYA\\OneDrive\\Desktop\\pip_Malya\\Python\\GEN\\theory\\dataset01\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content="\ufeffUnnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [101]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [102]:
embed = embedding_model.embed_query("Hi my name is Deepmalya")

In [103]:
print(len(embed))

384


This shows that different vector chunks has been created in the n-dimensional vector space for the text I placed.

In [104]:
print(embed[:5])

[-0.05844542384147644, -0.05370223522186279, -0.02370099537074566, -0.019565032795071602, -0.11788613349199295]


now  we want to store all the vector embeddings we created for the text into a vector db (here the implicit one supported by langchain). but this time let's noy automate this with the VectorstoreIndexCreator pipeline. let's manually do it ourselves for deeper understanding 


In [105]:
db = DocArrayInMemorySearch.from_documents(docs, embedding_model)
#this creates a vector store for the docs

In [106]:
query = "Please suggest a shirt with sunblocking"

In [107]:
docs = db.similarity_search(query)

In [115]:
len(docs) # there are 4 such similar answers to the same query

4

In [111]:
docs[0]

Document(metadata={'source': 'C:\\Users\\DEEPMALYA\\OneDrive\\Desktop\\pip_Malya\\Python\\GEN\\theory\\dataset01\\OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content='\ufeffUnnamed: 0: 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [114]:
'''now for all the similar documents found from the vector storeb against ourn query we need to filter out the best again.
This is why we first need to build a combined doc from the list of separated retrieved docs'''

document_unit="".join(docs[i].page_content for i in range(len(docs)))

In [119]:
response = model.invoke(f"{document_unit} Question: Please list all \
your shirts with sun protection in a table in markdown and summarize each one.") 

In [122]:
#  Now the response we got is a structured response having Ai message object, not a plain string. Thus to get the exact content we want we need to :
display(Markdown(response.content))

**Sun Protection Shirts Table**

| Name | Description | Fabric | UPF Rating | Features |
| --- | --- | --- | --- | --- |
| Sun Shield Shirt by | Blocks 98% of sun's harmful rays, high-performance fabric | 78% nylon, 22% Lycra Xtra Life | 50+ | Moisture-wicking, abrasion-resistant, quick-drying |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve shirt with superior SunSmart protection | 71% nylon, 29% polyester | 50+ | Wrinkle-resistant, moisture-wicking, front and back cape venting |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection, wrinkle-free and quickly evaporates perspiration | 52% polyester, 48% nylon | 50+ | Front and back cape venting, two front bellows pockets |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart protection | 71% nylon, 29% polyester | 50+ | Wrinkle-resistant, low-profile pockets, side shaping, front and back cape venting |

**Summary of Each Shirt:**

1. **Sun Shield Shirt by**: A high-performance sun shirt that blocks 98% of the sun's harmful rays, providing excellent protection and comfort.
2. **Tropical Breeze Shirt**: A lightweight, breathable long-sleeve shirt designed for fishing and travel, offering superior SunSmart protection and innovative features like wrinkle-resistant fabric and front and back cape venting.
3. **Men's Plaid Tropic Shirt, Short-Sleeve**: A comfortable and practical sun shirt designed for fishing and travel, featuring wrinkle-free fabric, front and back cape venting, and two front bellows pockets.
4. **Women's Tropical Tee, Sleeveless**: A stylish and functional sleeveless button-up shirt with SunSmart protection, featuring a flattering fit, wrinkle-resistant fabric, and innovative features like low-profile pockets and side shaping.

In [128]:
# this entire workflow can be guided and automated by the VectorstoreIndexCreator pipeline or by using Chains from langchain,
# we need to build a retriever that helps us with the Q&A for our documents.

retriever = db.as_retriever()

#We create a chain for the retrieval process passing it the parameters to work with:

qa=RetrievalQA.from_chain_type(llm=model,chain_type='stuff',retriever=retriever,verbose=True) #chain_type="stuff" says that stuff all the documents into a single context

In [125]:
query1="Please list all your shirts with sun protection in a table in markdown and summarize each one."

In [ ]:
response1=(query1) 
print(response1)



> Entering new RetrievalQA chain...

> Finished chain.
Here's a table of shirts with sun protection:

| Name | Description | Features |
| --- | --- | --- |
| Sun Shield Shirt by | High-performance sun shirt with UPF 50+ protection | Softly shapes the body, wicks moisture, abrasion resistant, handwash, line dry |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with SunSmart protection | Slightly fitted, shell: 71% nylon, 29% polyester, UPF 50+ rated, machine wash and dry |
| Sunrise Tee | Lightweight, high-performance UV-protective button down shirt | Slightly fitted, lightweight synthetic fabric, UPF 50+ rated, wrinkle-free, machine wash and dry |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve UPF shirt with SunSmart protection | Traditional fit, wrinkle-resistant and moisture-wicking fabric, UPF 50+ rated, machine wash and dry |

Here's a brief summary of each shirt:

* The Sun Shield Shirt by is a high-performance sun shirt that provides exc

# **05.Langchain - Evaluation**

## Outline:
Example generation

Manual evaluation (and debuging)

LLM-assisted evaluation

LangChain evaluation platform

We'll first create and then evaluate a Q&A application in this session

In [10]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
file = r'C:\Users\DEEPMALYA\OneDrive\Desktop\pip_Malya\Python\GEN\theory\dataset01\OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file,encoding='UTF-8')
data = loader.load()

In [11]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_25968\2537902316.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\DEEPMALYA\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Step 1: Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [30]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

groq_chat = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    temperature=0.1,    # Controls randomness in responses
    model_name="llama-3.1-8b-instant"   # or any Groq-supported model
)

**Storage logic (embeddings, indexing) lives in one module**

In [15]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch,embedding=embedding_model).from_loaders([loader])

**Retrieval + reasoning logic lives in another**

In [31]:
llm = groq_chat
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", # stuff for small scal document q&a
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

Evaluate using favourable data - points from the data given to model

In [21]:
data[10]

Document(metadata={'source': 'C:\\Users\\DEEPMALYA\\OneDrive\\Desktop\\pip_Malya\\Python\\GEN\\theory\\dataset01\\OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content="\ufeffUnnamed: 0: 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [22]:
data[11]

Document(metadata={'source': 'C:\\Users\\DEEPMALYA\\OneDrive\\Desktop\\pip_Malya\\Python\\GEN\\theory\\dataset01\\OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content='\ufeffUnnamed: 0: 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

**Hard Coded Examples - good queries as a test point from the data**

In [23]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [78]:
qa.run(examples)

AttributeError: 'dict' object has no attribute 'run'

**LLM generated examples**

In [79]:
from langchain.evaluation.qa import QAGenerateChain


In [80]:
example_gen_chain = QAGenerateChain.from_llm(groq_chat)

In [81]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_25968\3125874183.py:1: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_examples = example_gen_chain.apply_and_parse(


In [82]:
new_examples[0]

{'qa_pairs': {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it have to prevent odor buildup?",
  'answer': "The innersole of the Women's Campside Oxfords is made of EVA (Ethylene-Vinyl Acetate) material, and it features Cleansport NXT antimicrobial odor control."}}

In [83]:
data[0]


Document(metadata={'source': 'C:\\Users\\DEEPMALYA\\OneDrive\\Desktop\\pip_Malya\\Python\\GEN\\theory\\dataset01\\OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content="\ufeffUnnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [84]:
examples+=new_examples

In [85]:
qa.run(examples[0]["query"])

AttributeError: 'dict' object has no attribute 'run'

# **#Langchain Special - Runnables**

This defines how chains works internally ....**A deeper insight to Langchain framework**

**Runnables** - the engine that runs the chains and ultimately the entire langchain framework

While building a project with AI - say building a PDF content extractor, other than interacting with LLM APIs, there are several other aspects in the workkflow - the Document loaders, splitting data into chunks, converting them into embeddings , storing them in a db, retrieving the content, using LLM and NLP for the output, parsing it as a structured output to the user. This is where langchian decided to create different classes for these respective Operations so that building a project using this framework becomes easier

## Let's build a PDF Reader system

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import csv_loader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import openai

In [ ]:
# Load the document

loader=TextLoader("docx.text") # routes to the filepath
documents=loader.load() # reads the contents from the file

In [11]:
# Split the text into smaller chunks

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50) #chunk size defines the number of words to create an embedding for in a single go...while overlap defines the text continuity to preserve context
docs=text_splitter.split_documents(documents)


In [17]:
vector_store=Chroma.from_documents(docs,HuggingFaceBgeEmbeddings())

C:\Users\DEEPMALYA\AppData\Local\Temp\ipykernel_26096\3681082152.py:1: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  vector_store=Chroma.from_documents(docs,HuggingFaceBgeEmbeddings())


ValueError: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.